# Commodity Signal Scanner - CSV Test Version

This notebook tests the commodity signal scanner using CSV data instead of Bloomberg BQL.

**Purpose:** Debug and calibrate the signal scanner before deploying to BQuant.

**Signal Components:**
- Sentiment Score (40%) - Momentum/trend analysis with TD Sequential
- Correlation Alignment (25%) - Cross-asset confirmation
- Divergence Detection (20%) - Mean-reversion opportunities
- Regime Adjustment (15%) - Macro context

**Data Sources:**
- `combined_prices.csv` - Close prices for all assets
- `OHLV_*.csv` - OHLCV data for individual commodities

---
## 1. Setup and Imports

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 200)

# Import CSV data loader
from csv_data_loader import (
    load_combined_prices,
    load_ohlv_data,
    fetch_historical_data,
    fetch_price_data,
    fetch_multi_asset_data,
    get_available_tickers,
    validate_data,
    COMMODITY_UNIVERSE,
    CURRENCY_UNIVERSE,
    BOND_UNIVERSE,
    EQUITY_UNIVERSE,
)

# Import full signal generator
from commodity_signals_csv import (
    CommoditySignalGenerator,
    calculate_technical_indicators,
    calculate_td_sequential,
    calculate_sentiment_score,
    classify_signal,
    print_signal_report,
    SIGNAL_THRESHOLDS,
)

# Import correlation engine
from correlation_analysis_csv import (
    CorrelationEngine,
    EXPECTED_CORRELATIONS,
    KEY_CORRELATIONS,
    print_correlation_summary,
)

print("Imports successful!")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

---
## 2. Data Validation

Check that CSV data loads correctly and identify any issues.

In [ ]:
# Run data validation
print("Running data validation...")
print("=" * 60)

validation = validate_data()

print("\nCOMBINED PRICES SUMMARY:")
print(f"  Total rows: {validation['combined_prices'].get('total_rows', 'N/A')}")
print(f"  Date range: {validation['combined_prices'].get('date_range', 'N/A')}")

print("\nOHLV FILES SUMMARY:")
for ticker, info in validation['ohlv_files'].items():
    print(f"  {ticker}: {info['rows']} rows, {info['date_range']}")

print("\nISSUES FOUND:")
if validation['issues']:
    for issue in validation['issues']:
        print(f"  Warning: {issue}")
else:
    print("  No critical issues")

print("\nAVAILABLE TICKERS:")
available = get_available_tickers()
print(f"  OHLV data: {available['ohlv']}")
print(f"  Close data: {available['close']}")

---
## 3. Initialize Signal Generator

In [ ]:
# Initialize the signal generator
print("Initializing Commodity Signal Generator...")
print("=" * 60)

generator = CommoditySignalGenerator(lookback_days=120)

print("\nFetching price data for all assets...")
print("This may take a moment...")

---
## 4. Generate All Signals

In [ ]:
# Generate signals for all commodities
print("Generating signals for all commodities...")
print("=" * 60)

signals = generator.generate_all_signals()

# Print the full signal report
print_signal_report(signals)

---
## 5. Detailed Signal Breakdown

In [ ]:
# Show detailed breakdown for each commodity
print("DETAILED SIGNAL BREAKDOWN")
print("=" * 80)

for ticker, data in signals.items():
    name = COMMODITY_UNIVERSE.get(ticker, {}).get('name', ticker)
    
    print(f"\n{name} ({ticker})")
    print("-" * 40)
    print(f"  Final Signal: {data['signal']:.2f}")
    print(f"  Classification: {data['classification']}")
    print(f"  Action: {data['action']}")
    
    print(f"\n  Components:")
    comp = data.get('components', {})
    print(f"    Sentiment:   {comp.get('sentiment', 0):>6.2f} (weighted: {comp.get('sentiment_weighted', 0):>6.2f})")
    print(f"    Correlation: {comp.get('correlation', 0):>6.2f} (weighted: {comp.get('correlation_weighted', 0):>6.2f})")
    print(f"    Divergence:  {comp.get('divergence', 0):>6.2f} (weighted: {comp.get('divergence_weighted', 0):>6.2f})")
    print(f"    Regime Adj:  {comp.get('regime_adjustment', 0):>6.2f}")
    
    div = data.get('divergence_details', {})
    if div.get('divergence_type') != 'none':
        print(f"\n  Divergence: {div.get('divergence_type', 'none').upper()} (magnitude: {div.get('magnitude', 0):.2f})")

---
## 6. Macro Regime Analysis

In [ ]:
# Display macro regime
first_signal = list(signals.values())[0]
macro = first_signal.get('macro_regime', {})

print("MACRO REGIME ANALYSIS")
print("=" * 60)

print(f"\nCurrent Quadrant: {macro.get('quadrant', 0)} - {macro.get('name', 'Unknown')}")
print(f"Description: {macro.get('description', 'N/A')}")

print(f"\nGrowth Signal:    {macro.get('growth_signal', 0):>+.2f} ({'Rising' if macro.get('growth_rising') else 'Falling'})")
print(f"Inflation Signal: {macro.get('inflation_signal', 0):>+.2f} ({'Rising' if macro.get('inflation_rising') else 'Falling'})")

print("\nQuadrant Explanation:")
print("  Quad 1 (Goldilocks): Rising growth + Falling inflation - Risk-on, gold underperforms")
print("  Quad 2 (Reflation):  Rising growth + Rising inflation - Mixed")
print("  Quad 3 (Stagflation): Falling growth + Rising inflation - Gold shines")
print("  Quad 4 (Risk-Off):   Falling growth + Falling inflation - Safe haven demand")

---
## 7. Signal Visualization

In [ ]:
# Visualize signals
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Prepare data
names = [COMMODITY_UNIVERSE.get(t, {}).get('name', t) for t in signals.keys()]
signal_values = [s['signal'] for s in signals.values()]

# Sort by signal value
sorted_indices = np.argsort(signal_values)[::-1]
names_sorted = [names[i] for i in sorted_indices]
signals_sorted = [signal_values[i] for i in sorted_indices]

# Bar colors based on signal
colors = ['darkgreen' if s >= 7 else 'green' if s >= 4 else 'red' if s <= -7 else 'salmon' if s <= -4 else 'gray' 
          for s in signals_sorted]

# Left plot: Signal bars
ax1 = axes[0]
bars = ax1.barh(names_sorted, signals_sorted, color=colors)
ax1.axvline(x=0, color='black', linewidth=0.5)
ax1.axvline(x=4, color='green', linestyle='--', alpha=0.5, label='Bullish threshold')
ax1.axvline(x=-4, color='red', linestyle='--', alpha=0.5, label='Bearish threshold')
ax1.axvline(x=7, color='darkgreen', linestyle=':', alpha=0.5)
ax1.axvline(x=-7, color='darkred', linestyle=':', alpha=0.5)
ax1.set_xlabel('Signal Score', fontsize=11)
ax1.set_title('Commodity Signals', fontsize=13, fontweight='bold')
ax1.set_xlim(-10, 10)
ax1.grid(True, alpha=0.3, axis='x')

# Add value labels
for i, (name, val) in enumerate(zip(names_sorted, signals_sorted)):
    ax1.text(val + 0.2 if val >= 0 else val - 0.2, i, f'{val:.1f}', 
             va='center', ha='left' if val >= 0 else 'right', fontsize=10)

# Right plot: Component breakdown for Gold
ax2 = axes[1]
gold_signal = signals.get('GCA Comdty', {})
gold_comp = gold_signal.get('components', {})

comp_names = ['Sentiment', 'Correlation', 'Divergence', 'Regime']
comp_values = [
    gold_comp.get('sentiment_weighted', 0),
    gold_comp.get('correlation_weighted', 0),
    gold_comp.get('divergence_weighted', 0),
    gold_comp.get('regime_adjustment', 0)
]
comp_colors = ['steelblue', 'orange', 'purple', 'green']

bars2 = ax2.bar(comp_names, comp_values, color=comp_colors, alpha=0.8)
ax2.axhline(y=0, color='black', linewidth=0.5)
ax2.set_ylabel('Contribution to Signal', fontsize=11)
ax2.set_title(f'Gold Signal Breakdown (Total: {gold_signal.get("signal", 0):.2f})', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, val in zip(bars2, comp_values):
    ax2.text(bar.get_x() + bar.get_width()/2, val + 0.1 if val >= 0 else val - 0.3, 
             f'{val:.2f}', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

---
## 8. Correlation Analysis

In [ ]:
# Run correlation analysis
print("Running Correlation Analysis...")
print("=" * 60)

corr_engine = CorrelationEngine(lookback_days=120)

# Get Gold correlation summary
gold_corr_summary = corr_engine.get_correlation_summary('GCA Comdty')
print_correlation_summary(gold_corr_summary)

In [ ]:
# Correlation heatmap
print("Generating correlation heatmap...")

# Get all available tickers
combined = load_combined_prices(filter_valid=True)
all_tickers = [t for t in combined.columns if combined[t].notna().sum() > 50]
returns = fetch_multi_asset_data(all_tickers, days=120)

if returns is not None and len(returns.columns) > 0:
    corr_matrix = returns.corr()
    
    plt.figure(figsize=(14, 12))
    sns.heatmap(
        corr_matrix,
        annot=True,
        fmt='.2f',
        cmap='RdBu_r',
        center=0,
        vmin=-1,
        vmax=1,
        square=True,
        linewidths=0.5
    )
    plt.title('Cross-Asset Correlation Matrix (120-day)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("Not enough data for correlation matrix")

---
## 9. Cross-Commodity Ratios

In [ ]:
# Calculate cross-commodity ratios
print("Calculating cross-commodity ratios...")
print("=" * 60)

ratios = corr_engine.calculate_cross_commodity_ratios(days=120)

for ratio_name, df in ratios.items():
    if len(df) > 0:
        latest = df.iloc[-1]
        zscore = latest.get('Ratio_Zscore', 0)
        
        status = '⚠️ EXTREME' if abs(zscore) > 2 else '⚡ Elevated' if abs(zscore) > 1 else '✓ Normal'
        
        print(f"\n{ratio_name.replace('_', '/')}:")
        print(f"  Current Ratio: {latest['Ratio']:.2f}")
        print(f"  Z-Score: {zscore:+.2f} ({status})")
        print(f"  Mean: {df['Ratio'].mean():.2f}")
        print(f"  Range: {df['Ratio'].min():.2f} - {df['Ratio'].max():.2f}")

In [ ]:
# Plot Gold/Silver ratio
gs_ratio = ratios.get('Gold_Silver')

if gs_ratio is not None and len(gs_ratio) > 0:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    # Top: Ratio
    ax1.plot(gs_ratio['Date'], gs_ratio['Ratio'], 'navy', linewidth=1.5)
    ax1.axhline(y=gs_ratio['Ratio'].mean(), color='gray', linestyle='--', label='Mean')
    ax1.fill_between(gs_ratio['Date'], 
                     gs_ratio['Ratio'].mean() - 2*gs_ratio['Ratio'].std(),
                     gs_ratio['Ratio'].mean() + 2*gs_ratio['Ratio'].std(), 
                     alpha=0.2, color='blue', label='±2 Std Dev')
    ax1.set_ylabel('Gold/Silver Ratio')
    ax1.set_title('Gold/Silver Ratio Analysis', fontsize=13, fontweight='bold')
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)
    
    # Bottom: Z-Score
    ax2.plot(gs_ratio['Date'], gs_ratio['Ratio_Zscore'], 'purple', linewidth=1.5)
    ax2.axhline(y=0, color='gray', linestyle='-')
    ax2.axhline(y=2, color='red', linestyle='--', alpha=0.7, label='Overbought')
    ax2.axhline(y=-2, color='green', linestyle='--', alpha=0.7, label='Oversold')
    ax2.fill_between(gs_ratio['Date'], -2, 2, alpha=0.1, color='gray')
    ax2.set_ylabel('Z-Score')
    ax2.set_xlabel('Date')
    ax2.legend(loc='upper right')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(-3, 3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No Gold/Silver ratio data available")

---
## 10. Entry Signal Test

In [ ]:
# Test entry signals
print("ENTRY SIGNAL TEST")
print("=" * 60)

for ticker, info in COMMODITY_UNIVERSE.items():
    entry = generator.generate_entry_signal(ticker)
    
    print(f"\n{info['name']} ({ticker}):")
    print(f"  Entry Signal: {'YES' if entry['entry_signal'] else 'NO'}")
    if entry['entry_signal']:
        print(f"  Direction: {entry['entry_direction'].upper()}")
        print(f"  Confidence: {entry['confidence']:.0%}")
    print(f"  Reason: {entry['reason']}")
    print(f"  Correlation Regime: {entry['correlation_regime']}")

---
## 11. Technical Indicator Details

In [ ]:
# Show technical indicators for Gold
print("GOLD TECHNICAL INDICATORS")
print("=" * 60)

gold_data = fetch_historical_data('GCA Comdty', days=120)

if gold_data is not None:
    gold_indicators = calculate_technical_indicators(gold_data)
    gold_indicators = calculate_td_sequential(gold_indicators)
    
    latest = gold_indicators.iloc[-1]
    
    print(f"\nLatest Close: ${latest['Close']:,.2f}")
    print(f"\nMomentum (ROC):")
    print(f"  1-Day:  {latest['ROC_1']:+.2f}%")
    print(f"  5-Day:  {latest['ROC_5']:+.2f}%")
    print(f"  10-Day: {latest['ROC_10']:+.2f}%")
    print(f"  20-Day: {latest['ROC_20']:+.2f}%")
    
    print(f"\nMoving Averages:")
    print(f"  MA-5:  ${latest['MA_5']:,.2f}")
    print(f"  MA-10: ${latest['MA_10']:,.2f}")
    print(f"  MA-20: ${latest['MA_20']:,.2f}")
    print(f"  MA-50: ${latest['MA_50']:,.2f}")
    
    # MA alignment
    if latest['MA_5'] > latest['MA_10'] > latest['MA_20']:
        ma_status = "BULLISH ALIGNMENT"
    elif latest['MA_5'] < latest['MA_10'] < latest['MA_20']:
        ma_status = "BEARISH ALIGNMENT"
    else:
        ma_status = "MIXED"
    print(f"  Status: {ma_status}")
    
    print(f"\nRSI (14): {latest['RSI']:.1f}")
    if latest['RSI'] > 70:
        print("  Status: OVERBOUGHT")
    elif latest['RSI'] < 30:
        print("  Status: OVERSOLD")
    else:
        print("  Status: Neutral")
    
    print(f"\nTD Sequential:")
    print(f"  Buy Setup Count:  {int(latest['TD_Setup_Buy'])}")
    print(f"  Sell Setup Count: {int(latest['TD_Setup_Sell'])}")
    if latest['TD_Setup_Buy'] == 9:
        print("  ⚡ BUY 9 COMPLETE - Potential bottom")
    elif latest['TD_Setup_Sell'] == 9:
        print("  ⚡ SELL 9 COMPLETE - Potential top")
else:
    print("Failed to load Gold data")

In [ ]:
# Plot Gold price with indicators
if gold_data is not None and len(gold_indicators) > 20:
    fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)
    
    # Price and MAs
    ax1 = axes[0]
    ax1.plot(gold_indicators['Date'], gold_indicators['Close'], 'navy', linewidth=1.5, label='Close')
    ax1.plot(gold_indicators['Date'], gold_indicators['MA_5'], 'orange', linewidth=1, label='MA-5', alpha=0.8)
    ax1.plot(gold_indicators['Date'], gold_indicators['MA_20'], 'green', linewidth=1, label='MA-20', alpha=0.8)
    ax1.set_ylabel('Price')
    ax1.set_title('Gold - Price & Moving Averages', fontsize=12, fontweight='bold')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # RSI
    ax2 = axes[1]
    ax2.plot(gold_indicators['Date'], gold_indicators['RSI'], 'purple', linewidth=1.5)
    ax2.axhline(y=70, color='red', linestyle='--', alpha=0.7)
    ax2.axhline(y=30, color='green', linestyle='--', alpha=0.7)
    ax2.axhline(y=50, color='gray', linestyle='-', alpha=0.3)
    ax2.fill_between(gold_indicators['Date'], 30, 70, alpha=0.1, color='gray')
    ax2.set_ylabel('RSI')
    ax2.set_title('RSI (14)', fontsize=12, fontweight='bold')
    ax2.set_ylim(0, 100)
    ax2.grid(True, alpha=0.3)
    
    # TD Sequential
    ax3 = axes[2]
    ax3.bar(gold_indicators['Date'], gold_indicators['TD_Setup_Buy'], color='green', alpha=0.7, label='Buy Setup')
    ax3.bar(gold_indicators['Date'], -gold_indicators['TD_Setup_Sell'], color='red', alpha=0.7, label='Sell Setup')
    ax3.axhline(y=9, color='green', linestyle='--', alpha=0.5)
    ax3.axhline(y=-9, color='red', linestyle='--', alpha=0.5)
    ax3.set_ylabel('TD Count')
    ax3.set_title('TD Sequential Setup', fontsize=12, fontweight='bold')
    ax3.set_xlabel('Date')
    ax3.legend(loc='upper left')
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

---
## 12. Summary Report

In [ ]:
# Generate final summary
print("=" * 80)
print(f"{'COMMODITY SIGNAL SCANNER - FINAL REPORT':^80}")
print(f"{'Generated: ' + datetime.now().strftime('%Y-%m-%d %H:%M'):^80}")
print("=" * 80)

print("\nMACRO ENVIRONMENT:")
print(f"  Quadrant: {macro.get('name', 'Unknown')} (Q{macro.get('quadrant', 0)})")
print(f"  Growth: {'Rising' if macro.get('growth_rising') else 'Falling'} ({macro.get('growth_signal', 0):+.2f})")
print(f"  Inflation: {'Rising' if macro.get('inflation_rising') else 'Falling'} ({macro.get('inflation_signal', 0):+.2f})")

print("\nSIGNAL SUMMARY:")
print(f"  {'Commodity':<12} {'Signal':>8} {'Classification':<15} {'Action':<25}")
print("  " + "-"*65)

for ticker, data in sorted(signals.items(), key=lambda x: x[1]['signal'], reverse=True):
    name = COMMODITY_UNIVERSE.get(ticker, {}).get('name', ticker)
    print(f"  {name:<12} {data['signal']:>8.2f} {data['classification']:<15} {data['action']:<25}")

print("\nKEY RATIOS:")
for ratio_name, df in ratios.items():
    if len(df) > 0:
        latest = df.iloc[-1]
        zscore = latest.get('Ratio_Zscore', 0)
        status = 'EXTREME' if abs(zscore) > 2 else 'Elevated' if abs(zscore) > 1 else 'Normal'
        print(f"  {ratio_name.replace('_', '/'):<20}: {latest['Ratio']:.2f} (Z: {zscore:+.2f}) - {status}")

print("\nTRADING ACTIONS:")
for ticker, data in signals.items():
    name = COMMODITY_UNIVERSE.get(ticker, {}).get('name', ticker)
    entry = generator.generate_entry_signal(ticker)
    if entry['entry_signal']:
        print(f"  ➡️  {name}: {entry['entry_direction'].upper()} (Confidence: {entry['confidence']:.0%})")

# Check for any actionable signals
actionable = [s for s in signals.values() if abs(s['signal']) >= 4]
if not actionable:
    print("  No actionable signals at this time")

print("\n" + "=" * 80)

---
## Notes

### Issues Found During Testing
- Document any issues here

### Changes Needed for BQuant Version
- Replace `csv_data_loader` imports with BQL imports
- Replace `correlation_analysis_csv` with `correlation_analysis`
- Replace `commodity_signals_csv` with `commodity_signals`